# 层和块
:label:`sec_model_construction`

之前首次介绍神经网络时，我们关注的是具有单一输出的线性模型。
在这里，整个模型只有一个输出。
注意，单个神经网络
（1）接受一些输入；
（2）生成相应的标量输出；
（3）具有一组相关 *参数*（parameters），更新这些参数可以优化某目标函数。

然后，当考虑具有多个输出的网络时，
我们利用矢量化算法来描述整层神经元。
像单个神经元一样，层（1）接受一组输入，
（2）生成相应的输出，
（3）由一组可调整参数描述。
当我们使用softmax回归时，一个单层本身就是模型。
然而，即使我们随后引入了多层感知机，我们仍然可以认为该模型保留了上面所说的基本架构。

对于多层感知机而言，整个模型及其组成层都是这种架构。
整个模型接受原始输入（特征），生成输出（预测），
并包含一些参数（所有组成层的参数集合）。
同样，每个单独的层接收输入（由前一层提供），
生成输出（到下一层的输入），并且具有一组可调参数，
这些参数根据从下一层反向传播的信号进行更新。

事实证明，研究讨论“比单个层大”但“比整个模型小”的组件更有价值。
例如，在计算机视觉中广泛流行的ResNet-152架构就有数百层，
这些层是由*层组*（groups of layers）的重复模式组成。
这个ResNet架构赢得了2015年ImageNet和COCO计算机视觉比赛
的识别和检测任务 :cite:`He.Zhang.Ren.ea.2016`。
目前ResNet架构仍然是许多视觉任务的首选架构。
在其他的领域，如自然语言处理和语音，
层组以各种重复模式排列的类似架构现在也是普遍存在。

为了实现这些复杂的网络，我们引入了神经网络*块*的概念。
*块*（block）可以描述单个层、由多个层组成的组件或整个模型本身。
使用块进行抽象的一个好处是可以将一些块组合成更大的组件，
这一过程通常是递归的，如 :numref:`fig_blocks`所示。
通过定义代码来按需生成任意复杂度的块，
我们可以通过简洁的代码实现复杂的神经网络。

![多个层被组合成块，形成更大的模型](http://d2l.ai/_images/blocks.svg)
:label:`fig_blocks`

从编程的角度来看，块由*类*（class）表示。
它的任何子类都必须定义一个将其输入转换为输出的前向传播函数，
并且必须存储任何必需的参数。
注意，有些块不需要任何参数。
最后，为了计算梯度，块必须具有反向传播函数。
在定义我们自己的块时，由于自动微分（在 :numref:`sec_autograd` 中引入）
提供了一些后端实现，我们只需要考虑前向传播函数和必需的参数。

在构造自定义块之前，(**我们先回顾一下多层感知机**)
（ :numref:`sec_mlp_concise` ）的代码。
下面的代码生成一个网络，其中包含一个具有256个单元和ReLU激活函数的全连接隐藏层，
然后是一个具有10个隐藏单元且不带激活函数的全连接输出层。

In [3]:
import torch
from torch import nn
from torch.nn import functional as F#定义了一些函数，一些没有包括参数的一些函数

net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))#nn.Sequential定义了一种特殊的Module

X = torch.rand(2,20)#一个批量，给两个20维的向量

net(X)

tensor([[ 0.0441,  0.1836,  0.0693,  0.1147, -0.0625, -0.0748,  0.0575,  0.0870,
         -0.1349,  0.1336],
        [ 0.0240,  0.1013,  0.1543,  0.1009, -0.0523, -0.0031,  0.0363,  0.0658,
          0.0404,  0.0626]], grad_fn=<AddmmBackward>)

### 自定义一个块MLP，实现和上面一样的模型块

In [6]:
class MLP(nn.Module):
    #网络里面要的层，都在这里
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)
    #前向函数  
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))#把输入x放入hidden中,调用relu，放入输出
    

自定义块

In [7]:
net = MLP()
net(X)

tensor([[-0.0536,  0.0803, -0.0045,  0.0013, -0.1588,  0.1755, -0.0112, -0.0641,
          0.1367,  0.1616],
        [-0.0869,  0.0872, -0.0020, -0.0030, -0.1718,  0.1017,  0.0556, -0.1058,
          0.0182,  0.2378]], grad_fn=<AddmmBackward>)

顺序块

In [8]:
class MySequential(nn.Module):
    def __init__(self,*args):#传入参数 nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10)
        super().__init__()
        for block in args:#block按顺序拿到从前到后的层
            self._modules[block] = block#把这些层按顺序放进modules这个层里面
    
    def forward(self,X):
        for block in self._modules.values():#按顺序
            X = block(X)
        return X

net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[-0.1360,  0.0181, -0.0122, -0.2132,  0.1704,  0.0068,  0.0554, -0.0363,
          0.0641,  0.0251],
        [-0.0553,  0.1357, -0.0201, -0.1665,  0.1047, -0.2056,  0.1603,  0.0227,
          0.0407,  0.0387]], grad_fn=<AddmmBackward>)

请注意，`MySequential`的用法与之前为`Sequential`类编写的代码相同
（如 :numref:`sec_mlp_concise` 中所述）。

## [**在前向传播函数中执行代码**]

`Sequential`类使模型构造变得简单，
允许我们组合新的架构，而不必定义自己的类。
然而，并不是所有的架构都是简单的顺序架构。
当需要更强的灵活性时，我们需要定义自己的块。
例如，我们可能希望在前向传播函数中执行Python的控制流。
此外，我们可能希望执行任意的数学运算，
而不是简单地依赖预定义的神经网络层。

到目前为止，
我们网络中的所有操作都对网络的激活值及网络的参数起作用。
然而，有时我们可能希望合并既不是上一层的结果也不是可更新参数的项，
我们称之为*常数参数*（constant parameter）。
例如，我们需要一个计算函数
$f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$的层，
其中$\mathbf{x}$是输入，
$\mathbf{w}$是参数，
$c$是某个在优化过程中没有更新的指定常量。
因此我们实现了一个`FixedHiddenMLP`类，如下所示：


In [11]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
net(X)

tensor(-0.0660, grad_fn=<SumBackward0>)

我们可以[**混合搭配各种组合块的方法**]。
在下面的例子中，我们以一些想到的方法嵌套块。
*疯狂套娃*

In [27]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1320, grad_fn=<SumBackward0>)

1. 如果将`MySequential`中存储块的方式更改为Python列表，会出现什么样的问题？
>简而言之，_modules的主要优点是： 在模块的参数初始化过程中， 系统知道在_modules字典中查找需要初始化参数的子块。

2. 实现一个块，它以两个块为参数，例如`net1`和`net2`，并返回前向传播中两个网络的串联输出。这也被称为平行块。

In [26]:
class Paralles(nn.Module):
    def __init__(self):
        super().__init__()
        self.net1 = nn.Sequential(nn.Linear(20, 10))
        self.net2 = nn.Sequential(nn.Linear(20, 10))

    def forward(self, X):
        one = self.net1(X)
        two = self.net2(X)
        return torch.cat((one,two))
net = Paralles()
net(X)

tensor([[-0.2576,  0.4984, -0.0259,  0.3884,  0.4823,  0.2298, -0.1054,  0.1379,
          0.3945,  0.1236],
        [-0.2665,  0.3981, -0.1709,  0.3309,  0.4362, -0.1794, -0.0214,  0.2575,
          0.5980, -0.0678],
        [ 0.0271,  0.1219, -0.9192,  0.2837,  0.1473, -0.2107,  0.2581,  0.6597,
          0.0508,  0.7701],
        [-0.1257,  0.4043, -0.5683, -0.1146,  0.0117, -0.0702,  0.2947,  0.2350,
         -0.1586,  0.8593]], grad_fn=<CatBackward>)

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  2.7731e-01,
          2.4178e-01,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  1.0308e-01,  ...,  1.1567e-01,
          2.7966e-02,  7.3634e-03],
        [-8.0771e-01,  1.2782e-02,  7.0961e-04,  ...,  1.6663e-01,
         -4.3539e-01,  7.8132e-01],
        [-8.1357e-01,  2.2365e-01, -1.7543e-01,  ...,  4.8356e-01,
         -2.3926e-01,  6.9038e-01]], grad_fn=<CatBackward>)

3. 假设你想要连接同一网络的多个实例。实现一个函数，该函数生成同一个块的多个实例，并在此基础上构建更大的网络。

In [34]:
class monster(nn.Module):
    def __init__(self,module,mutation_nums):
        super().__init__()
        self.block = module
        self.nums = mutation_nums
        self.other = nn.Linear(16,20)

    def forward(self, X):
        for i in range(self.nums):
            X = self.block(X)
            X = self.other(X)
        return X

X = monster(chimera,3)
X  

monster(
  (block): Sequential(
    (0): NestMLP(
      (net): Sequential(
        (0): Linear(in_features=20, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=32, bias=True)
        (3): ReLU()
      )
      (linear): Linear(in_features=32, out_features=16, bias=True)
    )
    (1): Linear(in_features=16, out_features=20, bias=True)
    (2): FixedHiddenMLP(
      (linear): Linear(in_features=20, out_features=20, bias=True)
    )
  )
  (other): Linear(in_features=16, out_features=20, bias=True)
)